In [1]:
# Importa as bibliotecas
import pandas as pd
import numpy as np

In [2]:
pasta = 'Nada/'

In [3]:
# Faz a leitura dos dados
target = pd.read_excel(pasta + 'T_Conceicao.xlsx')

dados = pd.read_excel(pasta + 'agrupados.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'Nada/agrupados.xlsx'

In [38]:
coluna = 0
preenchidos = pd.DataFrame(index=target.index)
for item in dados.columns:
    modelo = pd.DataFrame(index=target.index)
    for linha in dados.index:
        modelo.loc[linha, dados.columns[coluna]] = dados.loc[linha, dados.columns[coluna]]
    
    repetir = False
    anterior = 0
    proximo = -1
    # Determina quais linhas são NA
    valores_na = modelo.isna()
    for linha in target.index:
        try:
            # Se a linha for NA
            if valores_na.iloc[linha].bool():
                # Se não estiver marcado para repetir o valor anterior
                if not repetir:
                    # Se não está na última linha
                    if linha < target.index.shape[0]:
                        # Busca o próximo valor na coluna
                        for busca_valor in range(linha+1, target.index.shape[0]):
                            # Se é encontrado um valor (não-NA)
                            if not valores_na.iloc[busca_valor].bool():
                                # Esse valor é definido como o 'próximo'
                                proximo = busca_valor
                                break
                    # Se existe um valor anterior à linha atual
                    if anterior != linha:
                        # Se existe um valor seguinte à linha atual (Busca próximo valor encontrou um resultado)
                        if proximo != -1:
                            # Define o valor atual como a média entre o anterior e o próximo
                            valor = np.mean([modelo.iloc[anterior,0], modelo.iloc[proximo,0]])
                        # Se não existe um valor seguinte à linha atual (Busca próximo valor não encontrou um resultado)
                        else:
                            # Repete o valor anterior
                            valor = modelo.iloc[anterior,0]
                    # Se não existe um valor anterior à linha atual
                    else:
                        # Se existe um valor seguinte à linha atual (Busca próximo valor encontrou um resultado)
                        if proximo != -1:
                            valor = modelo.iloc[proximo,0]
                        # Se não existe um valor seguinte à linha atual (Busca próximo valor não encontrou um resultado)
                        else:
                            print('Problema na linha', linha)

                # Atribui o valor à linha atual
                modelo.iloc[linha,0] = valor

                # Se o próximo valor não estiver na linha seguinte
                if proximo > linha + 1:
                    # Marca para repetir o valor atual
                    repetir = True
                # Se o próximo valor estiver na linha seguinte
                else:
                    repetir = False
                    proximo = -1

            anterior = linha
        except:
            pass
        
    preenchidos = pd.concat([preenchidos, modelo], axis = 1)
    coluna = coluna + 1
    
try:
    preenchidos.drop(index=-1, inplace=True)
except:
    pass

In [39]:
preenchidos.to_excel(pasta + 'ag_preenchidos.xlsx', 'Planilha1', index=False)